# data consistency

In [53]:
import json
import os
import pandas as pd
from typing import Dict, List
import re
import pickle
from tqdm import tqdm

# 강남구 - mango

In [2]:
gangnam = pd.read_csv('./Seoul/강남구.csv')
gangnam_not_in_bussiness = pd.read_csv('./Seoul/강남구_not_in_business.csv')
gangnam[gangnam['도로명전체주소'].isna()]
gangnam['소재지전체주소']=gangnam['소재지전체주소'].fillna('서울특별시 강남구 수서동 214-2')

In [3]:
mango = pickle.load(open('./output/seoul/강남구/mango_output.pkl','rb'))
kakao = pickle.load(open('./output/seoul/강남구/kakao_output.pkl','rb'))

In [4]:
c = 0
for i,j in mango.items():
    if j == [None]:
        c+=1

In [5]:
c

2827

## data 정합성 1
- bm25로 검색기 활용
- 학습 : mango의 지번 주소들만 활용
- query : 행안부의 지번 주소
- 결과 : mango의 지번 주소에서 해당 사업장명의 검색 결과(mango)에서 bm25 기준으로 sorting + 시군동이 유사할 경우만 고객에게 제시

In [162]:
# from rank_bm25 import BM25Okapi

In [187]:
# class CheckConstistency(object):
#     def __init__(self, mango_output:Dict, df:pd.DataFrame):
#         # df : 행안부 데이터
#         # mango_output : 행안부 데이터의 index가 key, value 는 mango에서 크롤링한 결과
#         self.mango_output = mango_output
#         self.df = df
#         self.bm25 = None
    
#     # 지번 주소 추출 
#     def get_jibun_address_in_mango(self):
#         output = []
#         c = 0
#         for i,j in self.mango_output.items():
#             for k in j:
#                 if k is None:
#                     continue
#                 else:
#                     tmp = k['detailed_info'].split('\n')
#                     assert tmp[1].startswith('지번'), tmp
#                     tmp = tmp[1]
#                     tmp = re.sub('지번 ','',tmp).strip().split()
#                     k['id']=c
#                     c+=1 # 주소가 중복일 수 있음.
#                     output.append(tmp)
#         return output
    
#     # bm 25 학습
#     def train_bm25(self):
#         tokenized_mango_jibun = self.get_jibun_address_in_mango()
#         self.bm25 = BM25Okapi(tokenized_mango_jibun)
    
#     # query 날리고, 해당 사업장명의 값들 중에서 sorting.
#     def retrieve(self,idx):
#         query = self.df['소재지전체주소']
#         if self.bm25 is None:
#             self.bm25 = self.train_bm25()
#         doc_scores = self.bm25.get_scores(query.strip().split())
#         sorted_idx = np.argsort(-doc_scores)
#         result = [(i,doc_scores[i]) for i in sorted_idx]
#         return result

In [188]:
#cc=CheckConstistency(mango_output,cur_gangnam)

## data 정합성 2
- anchor : 행안부의 지번 주소
- mango plate jibun 주소 와 비교해서 시구동세부동기준으로 동일한 경우만 남김.
- 남은 경우에 있어서는 자카드 유사도를 기준으로 sorting?

In [17]:
class CheckConstistency(object):
    def __init__(self, output:Dict, df:pd.DataFrame, threshold:int=3):
        # df : 행안부 데이터
        # mango_output : 행안부 데이터의 index가 key, value 는 mango에서 크롤링한 결과
        self.mango_output = output
        self.df = df
        self.bm25 = None
        self.threshold = threshold
        self.mango_outputs = {}
    
    # query 날리고, 해당 사업장명의 값들 중에서 sorting.
    def check(self,idx):
        anchor_jibun = self.df['소재지전체주소'][idx]
        #anchor_jibun = ' '.join(anchor_jibun.split()[1:self.threshold]).strip()
        anchor_jibun_addr1 = ' '.join(anchor_jibun.split()[1:self.threshold]).strip()
        anchor_jibun_addr2 = ' '.join(anchor_jibun.split()[1:self.threshold+1]).strip()
        
        mango1 = []
        mango2 = []
        for i in self.mango_output[idx]:
            try:
                tmp = i['detailed_info'].split('\n')
            except:
                continue
            if tmp[1].startswith('지번 '):
                tmp_jibun = re.sub('지번 ','',tmp[1])
                tmp_jibun_addr1 = ' '.join(tmp_jibun.split()[1:self.threshold]).strip()
                tmp_jibun_addr2 = ' '.join(tmp_jibun.split()[1:self.threshold+1]).strip()
                if anchor_jibun_addr1 == tmp_jibun_addr1:
                    mango1.append(i)
                if anchor_jibun_addr2 == tmp_jibun_addr2:
                    mango2.append(i)
            else:
                continue
        self.mango_outputs[idx]=dict(mango1=mango1, mango2=mango2)

In [18]:
len(gangnam)-2827

1239

In [22]:
cc = CheckConstistency(mango, gangnam)

In [23]:
for i in gangnam.index:
    cc.check(i)

In [24]:
c=0
for i,j in cc.mango_outputs.items():
    if j:
        if len(j['mango1'])!=len(j['mango2']):
            print(cc.df.loc[i])
            print()
            if j['mango1']:
                for k in j['mango1']:
                    print(k['detailed_info'])
                print()
                if j['mango2']:
                    for k in j['mango2']:
                        print(k['detailed_info'])
                else:
                    print(j['mango2'])

사업장명                                 씨유 역삼형지점
소재지전체주소            서울특별시 강남구 역삼동 797-16 명호빌딩 
도로명전체주소    서울특별시 강남구 논현로 327, 명호빌딩 지상1층 (역삼동)
자치구                                       강남구
Name: 10, dtype: object

주소 서울특별시 강남구 논현로80길 9
지번 서울시 강남구 역삼동 726-9
전화번호 02-558-4333
음식 종류 브런치 / 버거 / 샌드위치
가격대 만원 미만
영업시간 10:00 - 00:30
휴일 토, 일

[]
사업장명                                     행복한 김치
소재지전체주소              서울특별시 강남구 삼성동 159-7 현대백화점 
도로명전체주소    서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)
자치구                                         강남구
Name: 16, dtype: object

주소 서울특별시 강남구 선릉로112길 6 중건빌딩 1F
지번 서울시 강남구 삼성동 37-2 중건빌딩 1F
전화번호 02-543-8426
음식 종류 탕 / 찌개 / 전골
가격대 만원 미만
주차 무료주차 가능
영업시간 10:00 - 22:00

[]
사업장명                                     스타벅스 강남역신분당역사점
소재지전체주소                          서울특별시 강남구 역삼동 858 강남역 
도로명전체주소    서울특별시 강남구 강남대로 지하 396, 강남역 지하1층 2번,3번호 (역삼동)
자치구                                                 강남구
Name: 26, dtype: object

주소 서울특별시 강남구 테헤란로 231 센터필드
지번 서울시 강남구 역삼동 676 센터필드
전화번호 1522-323

In [27]:
#mango

In [65]:
kakao[2]

[{'name': '코코버블티 강남역점',
  'address': '서울 강남구 강남대로96길 9 1층',
  'score': '1.0',
  'reviews': ['룰루\n후기\n2\n별점평균\n1.0\n2022.11.18.\n진짜 너무너무너무너무너무맛이없음 살면서 이렇게 맛없는 버블티는 처음\n좋아요\n메뉴 더보기',
   '맑음\n후기\n23\n별점평균\n2.6\n2022.08.10.\n여자알바 오픈했냐고 물어보니 단답에 핸드폰만 보고있어요\n좋아요\n메뉴 더보기']}]

## 행안부 - mango 주소 mapping 

In [62]:
c = 0
for i in gangnam['사업장명']:
    if '커스텀커피' in i:
        print(i)
        c+=1


커스텀커피
커스텀커피 압구정점
커스텀커피 현대백화점 본점
커스텀커피 개포점
커스텀커피 대치점
커스텀커피 강남구청점
커스텀커피 개포래미안포레스트점


In [59]:
gangnam['사업장명'].apply(lambda i : '커스텀커

,사업장명,소재지전체주소,도로명전체주소,자치구
0,커스텀커피,서울특별시 강남구 세곡동 598,"서울특별시 강남구 헌릉로569길 35, 지상1층 101호 (세곡동)",강남구
1,송오브송즈로스터즈,서울특별시 강남구 신사동 569-8 압구정 디자인빌딩,"서울특별시 강남구 논현로163길 17, 압구정 디자인빌딩 지상1층 (신사동)",강남구
2,코코버블티 강남역점,서울특별시 강남구 역삼동 817-11,"서울특별시 강남구 강남대로96길 9, 지상1층 (역삼동)",강남구
3,KT그룹희망나눔재단,서울특별시 강남구 수서동 734 KT,"서울특별시 강남구 광평로47길 28, KT 지상6층 (수서동)",강남구
4,에스프레소엔투샷,서울특별시 강남구 논현동 77-9,"서울특별시 강남구 언주로135길 32-3, 지상1층 (논현동)",강남구
...,...,...,...,...
4061,씨유 수서럭키점,서울특별시 강남구 수서동 707번지 주공아파트 지상1층,"서울특별시 강남구 광평로51길 49, 지상1층 (수서동, 주공아파트)",강남구
4062,G PC방 대치점,서울특별시 강남구 대치동 912-23번지 우진빌딩,"서울특별시 강남구 삼성로 349, 우진빌딩 지하1층 (대치동)",강남구
4063,지에스25상록회관점,서울특별시 강남구 역삼동 699-27번지 청담빌딩,"서울특별시 강남구 언주로98길 25, 청담빌딩 지상1층 (역삼동)",강남구
4064,주식회사 청담은 롯데백화점 강남점,서울특별시 강남구 대치동 937번지 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지상6층 (대치동)",강남구


In [58]:
mango[0]

[{'score': '커스텀커피 (강남구청점)\n삼성동 - 카페 / 디저트\n263 1',
  'detailed_info': '주소 서울특별시 강남구 학동로68길 7 상가 1F 107호\n지번 서울시 강남구 삼성동 16 상가 1F 107호\n전화번호 02-548-0708\n음식 종류 카페 / 디저트\n가격대 만원 미만',
  'review_info': {'total': 1, 'good': 1, 'fine': 0, 'not_good': 0},
  'review': ['Festina Lente\n323\n19\n강남구청역 근처 자그마한 그래서 테이크아웃에 특화됐지만 매장 이용도 가능한 카페입니다. 2층으로 올라가는 계단이 좀 가파른 편이지만 올라가면 다른 분위기를 느낄 수 있습니다. 3~4개 정도의 테이블이 있는데 조용히 쉬다 갈 수 있기에는 충분했습니다. 당시 주문한 딸기라떼는 딸기우유에 퓨레가 첨가된 거였는데 맛이 없을 수 없는 식자재 조합이라서 딱히 더 할 말은 없습니다. 맛있었습니다. 그런데 이곳의 가격 전략인지는 모르겠는데 커피를 제외한 음료의 가격은 결코 가성비가 좋다고 말할 수는 없다고 봅니다.\n2022-06-16\n맛있다']},
 {'score': '커스텀커피 (개포점)\n개포/수서/일원 - 카페 / 디저트\n69 0',
  'detailed_info': '주소 서울특별시 강남구 영동대로 22 디에이치자이개포 단지내상가 808동 107호\n지번 서울시 강남구 일원동 743 디에이치자이개포 단지내상가 808동 107호\n전화번호 02-2226-5552\n음식 종류 카페 / 디저트\n가격대 만원 미만',
  'review_info': {},
  'review': []}]

In [43]:
anchor=gangnam['소재지전체주소'].tolist()

In [36]:
positive = []
for i,j in mango.items():
    for k in j:
        if k:
            tmp = k['detailed_info'].split('\n')
            tmp_doro = re.sub('주소 ','',tmp[0]).split()
            if tmp[1].startswith('지번 '):
                tmp_jibun = re.sub('지번 ','',tmp[1])
                positive.append(tmp_jibun)

In [48]:
mapping = {}
for a in tqdm(anchor):
    tmp_a = ' '.join(a.split()[1:3]).strip()
    answer = []
    for p in positive:
        tmp_p = ' '.join(p.split()[1:3]).strip()
        if tmp_a == tmp_p:
            answer.append(p)
    mapping[a]=answer

In [49]:
c=0
for i,j in mapping.items():
    if j:
        c+=1

In [51]:
c

3266

In [54]:
mapping = {}
for a in tqdm(anchor):
    tmp_a = ' '.join(a.split()[1:4]).strip()
    answer = []
    for p in positive:
        tmp_p = ' '.join(p.split()[1:4]).strip()
        if tmp_a == tmp_p:
            answer.append(p)
    mapping[a]=answer

100%|█████████████████████████████████████████████████████████████████████████████| 4066/4066 [00:16<00:00, 246.86it/s]


In [55]:
c=0
for i,j in mapping.items():
    if j:
        c+=1

In [56]:
c

417

In [ ]:
def post_processing(mango:Dict):
    

In [6]:
gangnam.loc[3345]

사업장명                                        커스텀커피 개포래미안포레스트점
소재지전체주소                      서울특별시 강남구 개포동 1282 개포 래미안 포레스트 
도로명전체주소    서울특별시 강남구 개포로 264, 상가1동 지하1층 102-2호 (개포동, 개포 래...
자치구                                                      강남구
Name: 3345, dtype: object

In [4]:

#mango[3345]

[{'score': '커스텀커피 (강남구청점)\n삼성동 - 카페 / 디저트\n250 1',
  'detailed_info': '주소 서울특별시 강남구 학동로68길 7 상가 1F 107호\n지번 서울시 강남구 삼성동 16 상가 1F 107호\n전화번호 02-548-0708\n음식 종류 카페 / 디저트\n가격대 만원 미만',
  'review_info': {'total': 1, 'good': 1, 'fine': 0, 'not_good': 0},
  'review': ['Festina Lente\n313\n19\n강남구청역 근처 자그마한 그래서 테이크아웃에 특화됐지만 매장 이용도 가능한 카페입니다. 2층으로 올라가는 계단이 좀 가파른 편이지만 올라가면 다른 분위기를 느낄 수 있습니다. 3~4개 정도의 테이블이 있는데 조용히 쉬다 갈 수 있기에는 충분했습니다. 당시 주문한 딸기라떼는 딸기우유에 퓨레가 첨가된 거였는데 맛이 없을 수 없는 식자재 조합이라서 딱히 더 할 말은 없습니다. 맛있었습니다. 그런데 이곳의 가격 전략인지는 모르겠는데 커피를 제외한 음료의 가격은 결코 가성비가 좋다고 말할 수는 없다고 봅니다.\n2022-06-16\n맛있다']},
 {'score': '커스텀커피 (개포점)\n개포/수서/일원 - 카페 / 디저트\n61 0',
  'detailed_info': '주소 서울특별시 강남구 영동대로 22 디에이치자이개포 단지내상가 808동 107호\n지번 서울시 강남구 일원동 743 디에이치자이개포 단지내상가 808동 107호\n전화번호 02-2226-5552\n음식 종류 카페 / 디저트\n가격대 만원 미만',
  'review_info': {},
  'review': []}]

In [15]:
c = 0
for i in kakao.values():
    if not i:
        c+=1


In [17]:
def get_kakao_review(data:Dict):
    for i,j in data.items():
        if j:
            for k in j:
                if k['reviews']:
                    for review in k['reviews']:
                        review = review.split('\n')
                        score = review[4]
                        date = review[5]
                        text = review[6]
                        k['processed_review'] = dict(score = score, date = date, text = text) 
        else:
            continue
    return kakao

In [15]:
kakao = get_kakao_review(kakao)

In [16]:
kakao

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def get_score(self, query):
    tokenized_query = self.tokenizer.tokenize(query)
    if self.passage_encoder is not None:
        pass
    else:
        self.passage_encoder = self.exec_embedding()
    doc_scores = self.passage_encoder.get_scores(tokenized_query)
    sorted_idx = np.argsort(-doc_scores)
    result = [(self.db_id_to_wiki_id[i],doc_scores[i]) for i in sorted_idx]
    return result

In [ ]:
def get_top_n(self,query,n):
    result = self.get_score(query)
    assert n<=self.doc_count, 'document의 길이가 n보다 작습니다.'
    output = result[:n]
    return output

In [159]:
tokenized_corpus = 
BM25Okapi()

NameError: name 'BM250kapi' is not defined

In [ ]:
tokenized_contexts = [self.tokenizer.tokenize(j['title']+' '+j['context']) if self.args.include_title else self.tokenizer.tokenize(j['context']) for i,j in tqdm(self.contexts.items(),desc='tokenize')]
        self.db_id_to_wiki_id = {i:j for i,j in enumerate(self.contexts.keys())}
        bm25 = BM25Okapi(tokenized_contexts) # use basic value
        

### check constistency
- 주소 기준 bm25 
- 이름 기준은 이미 망고플레이트 엔진이 진행했다고 가정

In [79]:
cur_gangnam = gangnam.loc[mango_crawling_index,:]

In [157]:
def check_consistency_by_address_mango(df:pd.DataFrame, mango_output:Dict, threshold = 1):
    # 시,군, 구 기준으로 지번 주소로 sorting
    output = {}
    c = 0
    for i,j in mango_output.items():
        output[i]=[]
        if j==[None]:
            continue
        for k in j:
            try:
                tmp = k['detailed_info'].split('\n')[1]
            except:
                c+=1
                continue
            if tmp.startswith('지번 '):
                jibun_address = re.sub('지번 ','',tmp).strip()
                jibun_address = ' '.join(jibun_address.split()[1:4]).strip()
                anchor_jibun_address = ' '.join(df['소재지전체주소'][i].split()[1:4]).strip()
                if jibun_address == anchor_jibun_address:
                    output[i].append(k)
                else:
                    continue
            else:
                continue
        output[i] = output[i][:threshold]
    print(c)
    return output

In [152]:
mango_output_checked=check_consistency_by_address_mango(cur_gangnam, mango_output)

26


In [156]:
for i,j in mango_output_checked.items():
    if len(j)>1:
        print(i)
        print(j)

3294
[{'score': '스타벅스 (코엑스몰)\n삼성동 - 카페 / 디저트\n2,924 5', 'detailed_info': '주소 서울특별시 강남구 영동대로 513 코엑스몰 아셈타워 후문방향 B1\n지번 서울시 강남구 삼성동 159 코엑스몰 아셈타워 후문방향 B1\n전화번호 02-758-8733\n음식 종류 카페 / 디저트\n가격대 만원 미만\n주차 유료주차 가능\n영업시간 07:00 - 23:00\n웹 사이트 식당 홈페이지로 가기', 'review_info': {'total': 5, 'good': 3, 'fine': 2, 'not_good': 0}, 'review': ['Tiffajy\n1193\n126\n머핀은 별로였으나 얼그레이시폰 진짜맛있어요..!!체인점 베이커리야 원래 기대감이 0인것도 있지만 얼그레이 향 진하게 나는 케익류 찾아보기 힘든데 스벅에서 찾음...ㅇㅅㅇ 촉촉한 얼그레이 케익 땡기면 찾아헤매지 말고 스벅으로 가세요ㅋㅋ!!\n2017-10-28\n괜찮다']}, {'score': '스타벅스리저브 (코엑스몰R점)\n삼성동 - 카페 / 디저트\n1,445 3', 'detailed_info': '주소 서울특별시 강남구 영동대로 513 코엑스몰 1F\n지번 서울시 강남구 삼성동 159 코엑스몰 1F\n전화번호 02-6002-3467\n음식 종류 카페 / 디저트\n가격대 만원 미만\n주차 유료주차 가능\n영업시간 07:00 - 23:00\n웹 사이트 식당 홈페이지로 가기', 'review_info': {'total': 3, 'good': 2, 'fine': 1, 'not_good': 0}, 'review': ['슈빙\n1715\n423\n여긴 항상 사람이 지이이이인짜 많음... 코엑스몰 안에 스벅 2개 있는데 둘다 사람 너무 많아서 주문줄부터 너무 긺.ㅠ 여기서 커피 사고 별마당 가서 책 읽는 코스 좋아하는데 피지오나 아메리카노처럼 손 안 가는 음료 시킬 땐 항상 시간이 아깝다.\n사이렌오더 꼭 하시길...\n2020-08-30\n괜찮다', '하

### result post_process

In [ ]:
# score
# detailed_info
# review

In [ ]:
class post_process(object):
    def __init__(self,result):
        self.result = result
    def pp_score(self):
        pass
    def pp_detailed_info(self):
        pass
    def pp_review(self):
        pass

In [ ]:
# data format
place_name
address_doro
address_jibun
geometry
properties
sources:List[Dict]
Dict - type, score, reviews(List(dit)]
                            date, review

In [77]:
gangnam.loc[1,:]

사업장명                                        송오브송즈로스터즈
소재지전체주소                서울특별시 강남구 신사동 569-8 압구정 디자인빌딩 
도로명전체주소    서울특별시 강남구 논현로163길 17, 압구정 디자인빌딩 지상1층 (신사동)
자치구                                               강남구
Name: 1, dtype: object

In [76]:
mango_output[1]

[{'score': '송오브송즈로스터즈\n신사/압구정 - 카페 / 디저트\n40 1',
  'detailed_info': '주소 서울특별시 강남구 논현로163길 17\n지번 서울시 강남구 신사동 569-8\n전화번호 0507-1310-9273\n음식 종류 카페 / 디저트\n가격대 만원 미만\n웹 사이트 식당 홈페이지로 가기',
  'review_info': {'total': 1, 'good': 1, 'fine': 0, 'not_good': 0},
  'review': ['르푸아송\n38\n2\n커피의 근본\n2022-08-17\n맛있다']}]

In [75]:
gangnam

,사업장명,소재지전체주소,도로명전체주소,자치구
0,커스텀커피,서울특별시 강남구 세곡동 598,"서울특별시 강남구 헌릉로569길 35, 지상1층 101호 (세곡동)",강남구
1,송오브송즈로스터즈,서울특별시 강남구 신사동 569-8 압구정 디자인빌딩,"서울특별시 강남구 논현로163길 17, 압구정 디자인빌딩 지상1층 (신사동)",강남구
2,코코버블티 강남역점,서울특별시 강남구 역삼동 817-11,"서울특별시 강남구 강남대로96길 9, 지상1층 (역삼동)",강남구
3,KT그룹희망나눔재단,서울특별시 강남구 수서동 734 KT,"서울특별시 강남구 광평로47길 28, KT 지상6층 (수서동)",강남구
4,에스프레소엔투샷,서울특별시 강남구 논현동 77-9,"서울특별시 강남구 언주로135길 32-3, 지상1층 (논현동)",강남구
...,...,...,...,...
4061,씨유 수서럭키점,서울특별시 강남구 수서동 707번지 주공아파트 지상1층,"서울특별시 강남구 광평로51길 49, 지상1층 (수서동, 주공아파트)",강남구
4062,G PC방 대치점,서울특별시 강남구 대치동 912-23번지 우진빌딩,"서울특별시 강남구 삼성로 349, 우진빌딩 지하1층 (대치동)",강남구
4063,지에스25상록회관점,서울특별시 강남구 역삼동 699-27번지 청담빌딩,"서울특별시 강남구 언주로98길 25, 청담빌딩 지상1층 (역삼동)",강남구
4064,주식회사 청담은 롯데백화점 강남점,서울특별시 강남구 대치동 937번지 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지상6층 (대치동)",강남구


In [ ]:
def check_consistency_1(data:dict):
    output = {}
    for i,j in data.items():
        output

In [8]:
gangnam['커스텀커피']

{'소재지전체주소': '서울특별시 강남구 세곡동 598 ',
 '도로명전체주소': '서울특별시 강남구 헌릉로569길 35, 지상1층 101호 (세곡동)',
 '자치구': '강남구',
 'query': '서울특별시 강남구 세곡동 커스텀커피',
 'kakao_score': [{'name': '커스텀커피 세곡점',
   'address': '서울 강남구 헌릉로569길 35 1층 101호',
   'score': '3.3',
   'reviews': ['스텔라\n후기\n63\n별점평균\n3.1\n2022.12.10.\n추천포인트\n맛\n친절\n흑임자라떼!!!\n꼭드세요\n좋아요\n메뉴 더보기',
    'Jm\n후기\n301\n별점평균\n3.0\n2022.09.18.\n영수증을 달라고 하면 혐오스러운 표정으로 바라보는 사장이 기자리고 있을 거임. 아마 영수증은 후기조작하는 지인들 전용인듯. 영수증 달라고 하는 자를 잠재적 비판리뷰어로 추측하는 한심한 마인드때문에 망할듯..\n좋아요\n메뉴 더보기',
    'PTLEE\n후기\n33\n별점평균\n3.1\n2022.08.17.\n말 그대로 인생 라떼 네요\n근래 먹은것중 젤 맛있어요\n좋아요\n메뉴 더보기']}]}

In [7]:
gangnam_output['서울 강남구 헌릉로569길 35 1층 101호']

{'name': '커스텀커피 세곡점',
 'address': '서울 강남구 헌릉로569길 35 1층 101호',
 'score': '3.3',
 'reviews': ['스텔라\n후기\n63\n별점평균\n3.1\n2022.12.10.\n추천포인트\n맛\n친절\n흑임자라떼!!!\n꼭드세요\n좋아요\n메뉴 더보기',
  'Jm\n후기\n301\n별점평균\n3.0\n2022.09.18.\n영수증을 달라고 하면 혐오스러운 표정으로 바라보는 사장이 기자리고 있을 거임. 아마 영수증은 후기조작하는 지인들 전용인듯. 영수증 달라고 하는 자를 잠재적 비판리뷰어로 추측하는 한심한 마인드때문에 망할듯..\n좋아요\n메뉴 더보기',
  'PTLEE\n후기\n33\n별점평균\n3.1\n2022.08.17.\n말 그대로 인생 라떼 네요\n근래 먹은것중 젤 맛있어요\n좋아요\n메뉴 더보기']}

## 정합성

In [200]:
mango_output[5]

[{'score': '얼린돌접시\n대치동 - 해산물 요리\n1,335 1',
  'detailed_info': '주소 서울특별시 강남구 삼성로58길 7\n지번 서울시 강남구 대치동 989-4\n전화번호 02-568-5292\n음식 종류 해산물 요리\n가격대 2만원-3만원\n주차 무료주차 가능\n영업시간 11:00 - 24:00',
  'review_info': {'total': 1, 'good': 1, 'fine': 0, 'not_good': 0},
  'review': ['김호진\n1\n0\n최고의 회입니다\n2018-01-05\n맛있다'],
  'id': 3},
 {'score': '얼린돌접시\n도곡동 - 해산물 요리\n1,196 0',
  'detailed_info': '주소 서울특별시 강남구 논현로26길 4\n지번 서울시 강남구 도곡동 448\n전화번호 02-571-2992\n음식 종류 해산물 요리\n가격대 2만원-3만원\n주차 무료주차 가능 공간협소\n영업시간 11:00 - 22:00\n휴일 일',
  'review_info': {},
  'review': [],
  'id': 4}]

In [204]:
for i,j in mango_output.items():
    for k in j:
        if k is None:
            continue
        if not k['detailed_info'].split('\n')[1].startswith('지번'):
            print(k)
            print(i)
            print()

{'score': '공차 (강남교보타워점)\n강남역 - 카페 / 디저트\n211 2', 'detailed_info': '주소 서울시 서초구 서초동 1303-16 아이파크에비뉴\n전화번호 02-536-7760\n음식 종류 카페 / 디저트\n가격대 만원 미만\n주차 주차공간없음\n영업시간 08:00 - 23:00\n마지막주문 22:30', 'review_info': {'total': 2, 'good': 1, 'fine': 1, 'not_good': 0}, 'review': ['Pearl\n473\n59\n뒷골목 잘 보이지 않는 곳에 있고, 아담하고 조금 낡아보이기도 하는 매장이지만, 메뉴는 타지점에 비해 정확히 레시피 대로 주는 느낌이이요. 체인점이 다 거기서 거기겠지만, 그래도 조금씩 차이가 있는데 다른 지점보다 좀 더 맛있어요. 덜 밍밍하고 덜 싱겁다고 해야하나..\n원래는 쩐주단을 더 좋아하는데 매장이 많이 사라져서 공차를 이용하는데, 최근 노량진점에서 먹었을때 설탕 30% 얼음 소량 신청해도 당도 50%로 인지 70%인지 모를 정도로 달게 타주고 얼음도 많이 넣어줘서 반정도 먹다가 버린 적이 있는데, 여기는 주문한 그대로 나와서 딱 맛있고 적당해요. 매장이 조금만 더 깔끔한 분위기면 좋겠어요^^;;\n2022-08-27\n괜찮다']}
3850

{'score': '더 로투스\n-\n0 0', 'detailed_info': '주소 서울 강남구 논현로167길 17 1층\n전화번호 02-547-5007', 'review_info': {}, 'review': []}
2839



# 강남구 - 카카오

In [3]:
kakao = json.load(open('./output/seoul/강남구/kakao.json'))

In [9]:
kakao_for_check = {i:j for i,j in kakao.items()}

In [10]:
kakao_crawling_index = []
kakao_output = {}
for i,j in kakao_for_check.items():
    # 사업장명이 2개라면 도로명 주소 기준으로 
    if len(gangnam.index[gangnam['사업장명']==i])==1:
        idx = gangnam.index[gangnam['사업장명']==i][0]
        kakao_output[idx] = j.get('kakao_score',[])
        kakao_crawling_index.append(idx)
    else:
        indice = gangnam.index[gangnam['사업장명']==i]
        gold_idx = None
        for idx in indice:
            # na 값인 지 확인
            if type(gangnam['소재지전체주소'][idx])==float:
                if gangnam['도로명전체주소'][idx].strip() == j['도로명전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
            else:
                if gangnam['소재지전체주소'][idx].strip() == j['소재지전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
        assert gold_idx is not None, (i,j)
        kakao_output[gold_idx]=j.get('kakao_score',[])
        kakao_crawling_index.append(idx)

In [12]:
further_crawling = []
for i in gangnam.index:
    if i not in kakao_crawling_index:
        further_crawling.append(i)

gangnam.loc[further_crawling,:].to_csv('./Seoul/강남구_kakao_further.csv')

In [13]:
len(further_crawling)

96